In [2]:
!pip install -q transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.9 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.4.0.6 which is incompatible.
torch 2.6.0+

In [3]:
import os
import pandas as pd

# Parse labels.txt
label_file = "/kaggle/input/labeldata/label.txt"
image_dir = "/kaggle/input/studydata/input/dataset"

data = []
with open(label_file, "r", encoding="utf-8") as f:
    for line in f:
        fname, text = line.strip().split("\t")
        img_path = os.path.join(image_dir, fname)
        if os.path.exists(img_path):
            data.append({"image_path": img_path, "text": text})

df = pd.DataFrame(data)


In [70]:
df.head(5)


,image_path,text
0,/kaggle/input/studydata/input/dataset/Picture1...,Why is it?
1,/kaggle/input/studydata/input/dataset/Picture1...,How are you?
2,/kaggle/input/studydata/input/dataset/Picture1...,Where are you going?
3,/kaggle/input/studydata/input/dataset/Picture1...,This is questionable.
4,/kaggle/input/studydata/input/dataset/Picture1...,Are you sure?


In [5]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")


2025-05-31 13:14:21.287097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748697261.504515      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748697261.575958      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from datasets import Dataset
from PIL import Image
import torch

# Convert DataFrame to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Preprocessing
def preprocess(example):
    image = Image.open(example["image_path"]).convert("RGB")
    pixel_values = processor(images=image, return_tensors="pt").pixel_values.squeeze(0)
    
    labels = processor.tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).input_ids.squeeze(0)
    labels[labels == processor.tokenizer.pad_token_id] = -100

    return {"pixel_values": pixel_values, "labels": labels}

# Map with proper formatting
dataset = dataset.map(preprocess, remove_columns=dataset.column_names)
dataset.set_format(type="torch")


Map:   0%|          | 0/86 [00:00<?, ? examples/s]

In [7]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    pixel_values = torch.stack([x["pixel_values"] for x in batch])
    labels = torch.stack([x["labels"] for x in batch])
    return {"pixel_values": pixel_values, "labels": labels}

train_dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)


In [8]:
from datasets import Dataset
from PIL import Image
import torch

In [9]:
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

In [55]:
from torch.optim import AdamW
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=5e-5)

best_loss = float("inf")  # Initialize with infinity

for epoch in range(100):  # Adjust as needed
    total_loss = 0
    model.train()
    
    for batch in train_dataloader:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} Average Loss: {avg_loss:.4f}")

    # Save best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        print(f"✅ New best model found at epoch {epoch+1} with loss {avg_loss:.4f}. Saving model...")
        model.save_pretrained("best-trocr-model")
        processor.save_pretrained("best-trocr-model")


Epoch 1 Average Loss: 0.3380
✅ New best model found at epoch 1 with loss 0.3380. Saving model...
Epoch 2 Average Loss: 0.3558
Epoch 3 Average Loss: 0.2544
✅ New best model found at epoch 3 with loss 0.2544. Saving model...
Epoch 4 Average Loss: 0.1165
✅ New best model found at epoch 4 with loss 0.1165. Saving model...
Epoch 5 Average Loss: 0.1051
✅ New best model found at epoch 5 with loss 0.1051. Saving model...
Epoch 6 Average Loss: 0.1782
Epoch 7 Average Loss: 0.1780
Epoch 8 Average Loss: 0.4440
Epoch 9 Average Loss: 0.2282
Epoch 10 Average Loss: 0.1371
Epoch 11 Average Loss: 0.2170
Epoch 12 Average Loss: 0.1921
Epoch 13 Average Loss: 0.1088
Epoch 14 Average Loss: 0.0927
✅ New best model found at epoch 14 with loss 0.0927. Saving model...
Epoch 15 Average Loss: 0.1640
Epoch 16 Average Loss: 0.2278
Epoch 17 Average Loss: 0.1174
Epoch 18 Average Loss: 0.1250
Epoch 19 Average Loss: 0.0448
✅ New best model found at epoch 19 with loss 0.0448. Saving model...
Epoch 20 Average Loss: 0.0725

In [11]:
model.save_pretrained("finetuned-trocr-handwriting")
processor.save_pretrained("finetuned-trocr-handwriting")


[]

In [69]:
model.eval()

test_image = Image.open("/kaggle/input/studydata/input/dataset/Picture4.jpg").convert("RGB")
pixel_values = processor(images=test_image, return_tensors="pt").pixel_values.to(device)

generated_ids = model.generate(pixel_values)
predicted_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Predicted Text:", predicted_text)


Predicted Text: This boring is.................


In [30]:
model_dir = "/kaggle/working/finetuned-trocr-handwriting"
model.save_pretrained(model_dir)
processor.save_pretrained(model_dir)


[]

In [31]:
from transformers import VisionEncoderDecoderModel, TrOCRProcessor
from PIL import Image
import torch

# Load your fine-tuned model and processor
model = VisionEncoderDecoderModel.from_pretrained("finetuned-trocr-handwriting")
processor = TrOCRProcessor.from_pretrained("finetuned-trocr-handwriting")


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

In [48]:
import cv2
import numpy as np
from PIL import Image
import os

def segment_lines(image_path):
    # Read and convert image to grayscale
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold to binary
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

    # Use dilation to connect text lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 10))
    dilated = cv2.dilate(thresh, kernel, iterations=2)

    # Find contours of lines
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours top to bottom
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[1])

    # Crop line images
    line_images = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        line_img = img[y:y+h, x:x+w]
        line_images.append(Image.fromarray(line_img))

    return line_images


In [66]:
line_images = segment_lines("/kaggle/input/studydata/input/dataset/Picture14.jpg")

full_text = ""
for line_img in line_images:
    pixel_values = processor(images=line_img, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    full_text += text + "\n"

print("Full Page Text:\n", full_text)


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor